In [1]:
import faiss

# DB 로딩
index = faiss.read_index("qa_index.faiss")

# 총 벡터 개수
print(f"✅ 총 벡터 수: {index.ntotal}")

✅ 총 벡터 수: 13234


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json

# 모델 로딩
model = SentenceTransformer("all-MiniLM-L6-v2")

# 벡터 DB 로딩
index = faiss.read_index("qa_index.faiss")

# 저장된 텍스트 청크도 같이 불러오기 (우리가 만들었던 chunks.json)
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# 쿼리 입력
query = "자살 하고 싶어요"
query_vec = model.encode([query], convert_to_numpy=True)
faiss.normalize_L2(query_vec)

# 검색 실행
D, I = index.search(query_vec, k=3)

# 결과 출력
for rank, idx in enumerate(I[0]):
    print(f"\n🔹 Top {rank+1}:")
    print(chunks[idx])


c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔹 Top 1:
Q: 요즘 힘들어요.
요즘에는 정말 기운이 없고 일상생활에서도 무기력감이 들어요. 무슨 일을 하던지 자꾸 힘들고 피곤한 느낌이 들어요. 이런 상태에서 일도 많고 스트레스 받는 상황인데 이게 어떻게 해결될지 모르겠어요.
A: 사우님 요즘에는 정말 기운이 없고 일상생활에서도 무기력감이 들어요. 무슨 일을 하던지 자꾸 힘들고 피곤한 느낌이 들어요. 이런 상태에서 일도 많고 스트레스 받는 상황인데 이게 어떻게 해결될지 모르겠다고 하셨네요.
사우님 힘든 상황이 계속 지속되면 우울증을 유발할 수도 있어요. 스트레스와 우울감은 서로 연관성이 높습니다. 스트레스를 많이 받으면 감정이 불안정해지고 자신감도 떨어지게 됩니다. 이러한 상황은 대인관계에서도 문제를 유발할 수 있어요. 그리고 스트레스가 지속되면 몸의 면역력도 떨어져서 몸이 약해지고 피로감이 들게 됩니다.
사우님 이럴 땐 일단 조금 쉬어보세요. 몸의 피로를 풀고 스트레스를 해소해야 하기 때문입니다. 스트레스를 완전히 풀어내는 방법은 없지만 스트레스를 관리하고 줄이는 방법은 있어요. 예를 들면, 규칙적인 생활 습관을 유지하는 것, 다양한 취미생활을 즐기는 것, 근육 이완법을 사용하는 것, 스트레스가 많이 쌓였을 때는 탄력적인 태도를 유지하는 것 등이 있습니다.
또한 스트레스를 다루기 위해서는 적극적인 대처가 필요합니다. 스트레스를 해소하기 위해 식습관을 바꾸거나, 운동을 하거나, 취미활동을 즐기거나, 지인과 함께하는 시간을 가지거나, 긍정적인 생각을 갖는 등의 방법이 있습니다. 자신에게 적합한 방법을 찾아서 스트레스를 관리해보세요.
사우님 지금은 일이나 학업, 가족관계, 대인관계 등 다양한 부분에서 스트레스를 받고 있는 것 같습니다. 하나씩 천천히 해결해 나가보세요. 혼자 해결하기 힘든 상황이라면 꼭 전문가의 도움을 받으세요. 정신건강전문 병원이나 상담센터에서 상담과 치료를 받아보세요. 상담을 통해 스트레스를 잘 다루고 우울증에 걸리지 않도록 도와드리겠습니다.
사우님 위의 답변이 조금이나마 도움이 되었

In [4]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Step 1: Load QA data
qa_pairs = []
with open("rag_data2.txt", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line.strip())
        qa_pairs.append({
            "question": item["input"].strip(),
            "answer": item["output"].strip()
        })

# Step 2: Extract Q and A separately
questions = [qa["question"] for qa in qa_pairs]
answers = [qa["answer"] for qa in qa_pairs]

# Step 3: Encode only the questions (Q)
model = SentenceTransformer("all-MiniLM-L6-v2")
question_embeddings = model.encode(questions, convert_to_numpy=True)
faiss.normalize_L2(question_embeddings)

# Step 4: Create FAISS index and add vectors
index = faiss.IndexFlatIP(question_embeddings.shape[1])
index.add(question_embeddings)

# Step 5: Save the index
faiss.write_index(index, "qa_question_only.faiss")

# Step 6: Save the answers separately (to match search index)
with open("qa_answers.json", "w", encoding="utf-8") as f:
    json.dump(answers, f, ensure_ascii=False, indent=2)

print("✅ 질문(Q)만 벡터화한 새로운 DB 생성 완료!")


✅ 질문(Q)만 벡터화한 새로운 DB 생성 완료!


In [5]:
# 필요한 모듈 로딩
import faiss
import json
from sentence_transformers import SentenceTransformer
import numpy as np

# 로딩
model = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.read_index("qa_question_only.faiss")
with open("qa_answers.json", "r", encoding="utf-8") as f:
    answers = json.load(f)

# 검색
query = "직장 스트레스 해결법이 궁금해요"
query_vec = model.encode([query], convert_to_numpy=True)
faiss.normalize_L2(query_vec)
D, I = index.search(query_vec, k=3)

# 답변 출력
for i, idx in enumerate(I[0]):
    print(f"\n🔍 Top {i+1} Answer:\n{answers[idx]}")



🔍 Top 1 Answer:
사우님은 학교에서 혹은 스타벅스에서 커피를 마시면서 공부를 하시면, 집중력이 떨어지며, 피곤해지신다는 것으로 추정됩니다. 이에 대한 해결방안에 대해서 같이 생각해보도록 합시다.
커피는 카페인이라는 화학물질이 함유되어있어서, 중추신경계에 자극을 줄 수 있습니다. 
커피를 마시면, 교감신경이 분비되어, 이로 인해 알람을 울리듯이 몸의 활동이 촉진되는데, 이로 인해 불안감이 높아지고 집중력이 떨어질 수 있습니다.
사우님은 커피를 많이 마시지 않는 것이 좋을 것입니다. 또한, 수면시간을 충분히 확보하고, 체력적으로도 충분히 휴식을 취하시는 것이 중요합니다. 
또한, 공부할 때, 스트레칭이나 간단한 운동을 함께하면, 몸을 움직이는 것에 의해 혈액순환을 좋게 하여 몸의 피로를 줄일 수 있습니다. 
마지막으로, 힘들 때마다 눈을 감고 심호흡을 해보시는 것도 좋습니다. 심호흡을 하면, 불안감을 줄일 수 있고, 몸의 균형을 잡아줄 수 있습니다.
이상입니다. 
사우님의 건강과 성공을 응원합니다.
감사합니다.

🔍 Top 2 Answer:
사우님은 요즘 회사 일이 너무 많아져서 힘들어 하시네요. 업무량이 많아지면서 사생활과 균형을 잡기가 어렵다는 건가요?
요즘에는 스마트워크가 적극적으로 도입되면서 업무와 생활의 경계가 모호해지고 균형을 잡기가 어려워졌습니다. 근무시간 외에도 스마트폰 등을 통한 전화와 이메일이 들어오면서 업무시간 외에도 계속해서 일과 관련된 메세지를 받게 되면서 과도한 업무 부담을 느끼게 되었습니다. 이는 업무와 사생활의 경계가 모호해지면서 스트레스를 유발하게 됩니다.
우선 일과 생활의 구분을 확실하게 해야 합니다. 업무가 끝나면 스마트폰이나 컴퓨터와 같은 업무용 장비를 사용하지 않는 것이 좋습니다. 또한 출근 시간에 늦지 않게 출근해서 일찍 퇴근하는 것이 좋습니다. 
그리고 퇴근 시간 이후에는 휴식을 충분히 취할 수 있도록 집중력을 높이는 운동, 걷기 등의 루틴이 좋습니다. 일과 생활의 경계를 구분하는 것이 중요하며, 주말이나 휴일